In [2]:
import gym
env = gym.make('MountainCar-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
from keras.models import Sequential
from keras.layers import Dense
from collections import deque
import random
from keras.optimizers import Adam

import numpy as np


class DQNAgent:
    
    def __init__(self, env, replay_size=1000, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995, gamma=0.99):
        self.state_size = env.observation_space.shape[0]
        self.num_actions = env.action_space.n
        self.model = self.build_model()
        self.replay_buffer = deque(maxlen=replay_size)
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.gamma = gamma

        
    def build_model(self):
        model = Sequential()
        model.add(Dense(100, input_dim=self.state_size, activation='relu'))
        model.add(Dense(100, activation='relu'))
        model.add(Dense(self.num_actions, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=0.001))
        return model
        
    def action(self, state):
        if np.random.random() <= self.epsilon:
            return np.random.randint(self.num_actions)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])
    
    def train_batch_from_replay(self, batch_size):
        if len(self.replay_buffer) < batch_size:
            return False
        minibatch = random.sample(self.replay_buffer, batch_size)
        batch_X = np.zeros((batch_size, self.state_size))
        batch_next_state = np.zeros((batch_size, self.state_size))
        for i, (state, action, reward, next_state, done) in enumerate(minibatch):
            batch_X[i] = state[0]
            batch_next_state[i] = next_state[0]
        target_Qs = self.model.predict(batch_X)
        batch_next_Qs = self.model.predict(batch_next_state)
        for i, (state, action, reward, next_state, done) in enumerate(minibatch):
            target = reward
            if not done:
                next_Qs = batch_next_Qs[i]
                target += self.gamma * np.max(next_Qs)
            target_Qs[i][action] = target
        self.model.fit(batch_X, target_Qs, epochs=1, verbose=0)
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def add_to_replay_buffer(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

Using TensorFlow backend.


In [4]:
agent = DQNAgent(env)

In [27]:
from tqdm import tqdm

done = False
batch_size = 32
num_episodes = 1000

for episode in tqdm(range(num_episodes)):
    state = env.reset()
    state = np.reshape(state, [1, agent.state_size])
    total_reward = 0
    
    for t in range(200):
        action = agent.action(state)
        next_state, reward, done, _ = env.step(action)
        if done:
            reward = 200
        total_reward += reward
        next_state = np.reshape(next_state, [1, agent.state_size])
        agent.add_to_replay_buffer(state, action, reward, next_state, done)
        state = next_state    
        agent.train_batch_from_replay(batch_size)
        if done:
            break
    print(total_reward)

  0%|          | 1/1000 [00:01<18:04,  1.09s/it]

46.0


  0%|          | 2/1000 [00:01<16:14,  1.02it/s]

86.0


  0%|          | 3/1000 [00:02<14:48,  1.12it/s]

...

114.0


100%|█████████▉| 999/1000 [15:08<00:00,  1.17it/s]

22.0


100%|██████████| 1000/1000 [15:09<00:00,  1.15it/s]

57.0


In [5]:
from keras.models import load_model
agent.model = load_model('rl_model.h5')

In [6]:
agent.epsilon = 0
for _ in range(10):
    state = env.reset()
    state = np.reshape(state, [1, agent.state_size])
    total_reward = 0
    for t in range(200):
        action = agent.action(state)
        next_state, reward, done, _ = env.step(action)
        total_reward += reward
        state = np.reshape(next_state, [1, agent.state_size])
        env.render()
        if done:
            break
    print(total_reward)
agent.epsilon = 0.5

-154.0
-153.0
-100.0
-84.0
-98.0
-189.0
-200.0
-92.0
-200.0
-149.0


In [30]:
agent.model.save('rl_model.h5')